# Third notebook on dermoscopic images analysis

One of the most popular approaches to defining sharpness/bluriness of an images is to apply Laplacian operator which is an approximation of second derivative over discrete 2D matrix. As the result we have near 0 values around edges in the original image, therefore variance of the result can give us an estimate of how well the edges are defined in the image.

This approach works quite well in the regular images analysis and is widely used in computer vision application. However to compute the lapalcian we have to convert an RGB image into gray-scale space, loosing color information. Moreover human skin is quite smooth and lack many pronounced details, leading to significantly fewer well defined edges in any dermoscopic image.

That's why I showed in the first notebook that we can't use this simple approach of analyzing variance of laplacian operator over gray version of the dermoscopic images of the leisons. In the absolute majority of the images the variance is too low to set a definitive threshold on which to base the decision.

Therefore lets analyze these images in different color spaces and estimate **'sharpness index'** per each channel (still using variance of the laplacian operator). 
**The higher the number of the index the better** since it is a reflection of how many edges are there in the image.
We'll analyze the images by separate channels, plot the histograms, and display basic descriptive analytics.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
from IPython.display import display_html
from PIL import Image
import gc
import cv2
from skimage.exposure import is_low_contrast, rescale_intensity
import random
from matplotlib import cm

# Directory
directory = '../input/siim-isic-melanoma-classification'
IMAGE_PATH = "../input/siim-isic-melanoma-classification/"
img_dir = IMAGE_PATH+'/jpeg/train'

# Import the 2 csv s
train_df = pd.read_csv(directory + '/train.csv')
print(f'There are {train_df.shape[0]} records in the train set')

random.seed(1970)

Split data into benign and malignant leisons subsets. Pick 9 random samples from each for furthere analysis.

In [ ]:
benign = train_df[train_df['benign_malignant']=='benign']
malignant = train_df[train_df['benign_malignant']=='malignant']
print(f'There are {benign.shape[0]} records in the benign set, and {malignant.shape[0]} in the malignant dataset')

random9_mal = [np.random.choice(malignant['image_name'].values+'.jpg') for i in range(9)]
random9_ben = [np.random.choice(benign['image_name'].values+'.jpg') for i in range(9)]

In [ ]:
print(random9_mal)
print(random9_ben)

'ISIC_2724529.jpg'

['ISIC_2607827.jpg', 'ISIC_2027582.jpg', 'ISIC_0337631.jpg', 'ISIC_2481044.jpg', 'ISIC_7503950.jpg', 'ISIC_6251753.jpg', 'ISIC_0961235.jpg', 'ISIC_8672524.jpg', 'ISIC_2724529.jpg']
['ISIC_3193768.jpg', 'ISIC_2751781.jpg', 'ISIC_6197292.jpg', 'ISIC_6641336.jpg', 'ISIC_0396641.jpg', 'ISIC_3316232.jpg', 'ISIC_6080436.jpg', 'ISIC_3236507.jpg', 'ISIC_3526268.jpg']

In [ ]:
def variance_of_laplacian(image):
# compute the Laplacian of the image and then return the focus
# measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(image, cv2.CV_64F).var()

def laplacian(image):
# compute the Laplacian of the image and then return the focus
# measure, which is simply the variance of the Laplacian
    res = cv2.Laplacian(image, cv2.CV_64F)
    lap_var = res.var()
    return res, lap_var

# show 9 images with Blur Index in 3*3 grid
def show9(images, l_type):
    print(f'{l_type} images with variance of laplacian of gray version of the images as Sharpness Index')

    # Adjust the size of your images
    plt.figure(figsize=(10,8))

    # Iterate and plot random images
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        img = plt.imread(os.path.join(img_dir, images[i]))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        fm = variance_of_laplacian(gray)
        plt.title(f'Sharpness Index : {fm:.2f}')
        plt.imshow(img, cmap='gray')
        plt.axis('off')

    # Adjust subplot parameters to give specified padding
    plt.tight_layout()
    
def show9_RGB(images, l_type):
    print(f'Display {l_type} Images')
    N_subplots = 6
    color = ('b','g','r')

    # Iterate and plot random images
    for i in range(9):
        fig = plt.figure()
        fig.set_figwidth(20)
        fig.set_figheight(3)
        img = plt.imread(os.path.join(img_dir, images[i]))
        ax = fig.add_subplot(1, N_subplots, 1)
        imgplot = plt.imshow(img)
        ax.set_title(f'Size: {img.shape}')
        plt.axis('off')

        for i,col in enumerate(color):
            ax = fig.add_subplot(1, N_subplots, i+2)
            imgplot = plt.imshow(img[:,:,i], cmap='gray')
            ax.set_title(col)
            plt.axis('off')

        color = ('b','g','r')
        ax = fig.add_subplot(1, N_subplots, 5)
        for i,col in enumerate(color):
            histr = cv2.calcHist([img],[i],None,[256],[0,255]) #excluding pure white 255 pixels of the borders in the images
            plt.plot(histr,color = col)
            plt.xlim([0,256])

        ax = fig.add_subplot(1, N_subplots, 6)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mat, fm = laplacian(gray)
        ax.set_title(f'Bluriness index : {fm:.2f}')
        plt.hist(mat.ravel(), bins=100) 
        plt.show()    
        
# pass an image in BGR format as an input
def BGR2MI(img):
    MI = 100 * np.log10(1/(img[:,:,2].astype('float')+1))
    MI_norm = (MI * 0.0042 + 1) * 255
    MI_s = MI + abs(MI.min())
    MI_rescaled = 255 * (MI_s / abs(MI_s.max()))
    return MI.astype('int16'), MI_rescaled.astype('int16'), MI_norm.astype('int16')
#     return MI.astype('int'), MI_rescaled.astype('int')

# pass an image in BGR format as an input
def BGR2EI(img):
    EI = 100 * (np.log10(1/(img[:,:,1].astype('float')+1)) - 1.44 * np.log10(1/(img[:,:,2].astype('float')+1)))
    EI_norm = (EI * 0.0017 + 0.4098) * 255
    EI = EI + abs(EI.min())
    EI_rescaled = 255 * (EI / abs(EI.max()))
    return EI.astype('int16'), EI_rescaled.astype('int16'), EI_norm.astype('int16')                
#   return EI.astype('int'), EI_rescaled.astype('int')                

## Melanin Index and Erythema Index

In this notebook I want to introduce Melanin Index and Erythema Index. I believe we can use it as for image quality analysis and so for our future lesion classifier.

In [ ]:
def show_image_analytics(images, l_type):
    print(f'{l_type} Images')
    N_rows = 4
    N_cols = 4
    HSV_color = ('H','S','V')
    BGR_color = ('b','g','r')
    YCrCb_color = ('Y','Cr','Cb')
    plot_color = ('r','g','b')
    pcts = [5, 25, 50, 75, 95]
    blur_idxs = {'r': 0, 'g': 0, 'b': 0, 'H': 0, 'S': 0, 'V': 0, 'Y': 0, 'Cr': 0, 'Cb': 0, 'MI': 0, 'EI': 0, 'Gray': 0}


    # Iterate and plot random images
    for n_img in range(len(images)):
        fig = plt.figure()
        fig.set_figwidth(20)
        fig.set_figheight(14)
        img_BGR = cv2.imread(os.path.join(img_dir, images[n_img]))
        ax = fig.add_subplot(N_rows, N_cols, 1)
        imgplot = plt.imshow(cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB))
        ax.set_title(f'Size: {img_BGR.shape}')
        plt.axis('off')

        MI, MI_rescaled, MI_norm = BGR2MI(img_BGR)
        ax = fig.add_subplot(N_rows, N_cols, 2)
        imgplot = plt.imshow(MI_norm, cmap='gist_heat')
#         imgplot = plt.imshow(MI_norm, cmap='Reds')
#         imgplot = plt.imshow(MI, cmap='gray')
#         imgplot = plt.imshow(MI_rescaled, cmap='gray')
        ax.set_title(f'Melanin Index')
        plt.axis('off')
        
        EI, EI_rescaled, EI_norm = BGR2EI(img_BGR)      
        ax = fig.add_subplot(N_rows, N_cols, 3)
        imgplot = plt.imshow(EI_norm, cmap='gist_heat')
#         imgplot = plt.imshow(EI, cmap='gray')
#         imgplot = plt.imshow(EI_rescaled, cmap='gray')
        ax.set_title(f'Erythema Index')
        plt.axis('off')
        
#         ax = fig.add_subplot(N_rows, N_cols, 4)
# #         imgplot = plt.imshow(MI_norm, cmap='gray')
#         imgplot = plt.imshow(EI_norm, cmap='gist_heat')
# #         imgplot = plt.imshow(EI_rescaled, cmap='gray')
#         ax.set_title(f'Erythema Index')
#         plt.axis('off')
        
        blur_idxs['MI'] = variance_of_laplacian(MI_norm)
        blur_idxs['EI'] = variance_of_laplacian(EI_norm)
        
        gray = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
        ax = fig.add_subplot(N_rows, N_cols, 4)
        mat, blur_idxs['Gray'] = laplacian(gray)
        ax.set_title('Laplacian of gray image')
        plt.hist(mat.ravel(), bins=100) 

        for i,col in enumerate(BGR_color):
            ax = fig.add_subplot(N_rows, N_cols, N_cols + i + 1)
            imgplot = plt.imshow(img_BGR[:,:,i], cmap='gray')
            ax.set_title(col)
            plt.axis('off')
            blur_idxs[col] = variance_of_laplacian(img_BGR[:,:,i])

        ax = fig.add_subplot(N_rows, N_cols, N_cols * 1 + 4)
        ax.set_title(f'Histograms of {BGR_color}')
        for i,col in enumerate(BGR_color):
            histr = cv2.calcHist([img_BGR],[i],None,[256],[0,255]) #excluding pure white 255 pixels of the borders in the images
            plt.plot(histr,color = col, label = BGR_color[i])
            plt.xlim([0,256])
        ax.legend()

        img = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
#         HSV_pcts = np.percentile(img, pcts)
        HSV_pcts = np.percentile(img[:,:,2], pcts)
        for i,col in enumerate(HSV_color):
            ax = fig.add_subplot(N_rows, N_cols, N_cols * 2 + i + 1)
            imgplot = plt.imshow(img[:,:,i], cmap='gray')
            ax.set_title(col)
            plt.axis('off')
            blur_idxs[col] = variance_of_laplacian(img[:,:,i])

        ax = fig.add_subplot(N_rows, N_cols, N_cols * 2 + 4)
        ax.set_title(f'Histograms of {HSV_color}')
        for i,col in enumerate(['pink','gold','indigo']):
            histr = cv2.calcHist([img],[i],None,[256],[0,255]) #excluding pure white 255 pixels of the borders in the images
            plt.plot(histr,color = col, label = HSV_color[i])
            plt.xlim([0,256])
        ax.legend()

        img = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2YCR_CB)
        YCrCb_pcts = np.percentile(img[:,:,0], pcts)
#         YCrCb_pcts = np.percentile(img, pcts)
        for i,col in enumerate(YCrCb_color):
            ax = fig.add_subplot(N_rows, N_cols, N_cols * 3 + i + 1)
            imgplot = plt.imshow(img[:,:,i], cmap='gray')
            ax.set_title(col)
            plt.axis('off')
            blur_idxs[col] = variance_of_laplacian(img[:,:,i])

        ax = fig.add_subplot(N_rows, N_cols, N_cols * 3 + 4)
        ax.set_title(f'Histograms of {YCrCb_color}')
        for i,col in enumerate(['coral', 'olive', 'maroon']):
            histr = cv2.calcHist([img],[i],None,[256],[0,255]) #excluding pure white 255 pixels of the borders in the images
            plt.plot(histr,color = col, label = YCrCb_color[i])
            plt.xlim([0,256])
        ax.legend()

        plt.show()

        BGR_pcts = np.percentile(img_BGR, pcts)
        print(f'Percentiles of intensities in RGB space: {dict(zip(pcts,BGR_pcts))}')
        print(f'Percentiles of intensities in HSV space: {dict(zip(pcts,HSV_pcts))}')
        print(f'Percentiles of intensities in YCrCb space: {dict(zip(pcts,YCrCb_pcts))}')
        print('Sharpness Index: ')
        bis = ''
        for k in blur_idxs:
            bis = bis + k + f': {blur_idxs[k]:.2f}  '
        print(bis)
        pcts_contrast = [1, 5, 25, 50, 75, 95, 99]
        gray_pcts = np.percentile(gray, pcts_contrast)
        contrast_clipped = (gray_pcts[-1] - gray_pcts[0])/255
        contrast = (gray.max() - gray.min())/255
        print(f'Full contrast: {contrast:.2f}; clipped (1%-99%) contrast: {contrast_clipped:.2f}') #remove outliers
        
        print('-'*80)

In [ ]:
# show_image_analytics(random9_ben[:2], 'Benign')
show_image_analytics(random9_ben, 'Benign')

In [ ]:
random9_mal

In [ ]:
show_image_analytics(random9_mal, 'Malignant')
# show_image_analytics(random9_mal[:4], 'Malignant')

'ISIC_2724529.jpg'


In [ ]:
show9_RGB(random9_mal, 'Malignant')

In [ ]:
def show_image_EI_MI(images, l_type):
#     print(f'{l_type} Images')
    N_rows = 4
    N_cols = 4
    BGR_color = ('b','g','r')


    # Iterate and plot random images
    for n_img in range(len(images)):
        fig = plt.figure()
        fig.set_figwidth(20)
        fig.set_figheight(14)
        img_BGR = cv2.imread(os.path.join(img_dir, images[n_img]))
#         print(images[n_img])
        ax = fig.add_subplot(N_rows, N_cols, 1)
        imgplot = plt.imshow(cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB))
        ax.set_title(f'{l_type} : {images[n_img]}')
#         ax.set_title(f'Size: {img_BGR.shape}')
        plt.axis('off')

        MI, MI_rescaled, MI_norm = BGR2MI(img_BGR)
        ax = fig.add_subplot(N_rows, N_cols, 2)
#         imgplot = plt.imshow(MI_norm, cmap='gist_heat')
#         imgplot = plt.imshow(MI_norm, cmap='copper')
        imgplot = plt.imshow(MI_norm, cmap='gray')
        ax.set_title(f'Melanin Index')
        plt.axis('off')
        plt.colorbar(shrink = 0.8)
#         plt.colorbar(surf, cax=cax, shrink=0.1, aspect=5)

        
        EI, EI_rescaled, EI_norm = BGR2EI(img_BGR)      
        ax = fig.add_subplot(N_rows, N_cols, 3)
        imgplot = plt.imshow(EI_norm, cmap='gray')
#         imgplot = plt.imshow(EI_norm, cmap='gist_heat')
        ax.set_title(f'Erythema Index')
        plt.axis('off')
        plt.colorbar(shrink = 0.8)
        

        ax = fig.add_subplot(N_rows, N_cols, 4)
        ax.set_title(f'Histograms of {BGR_color}')
        for i,col in enumerate(BGR_color):
            histr = cv2.calcHist([img_BGR],[i],None,[256],[0,255]) #excluding pure white 255 pixels of the borders in the images
            plt.plot(histr,color = col, label = BGR_color[i])
            plt.xlim([0,256])
        ax.legend()

        plt.show()



In [ ]:
random9_mal[5]

In [ ]:
show_image_EI_MI(random9_mal[5:6], 'Malignant')

In [ ]:
show_image_EI_MI(random9_mal[5:6], 'Malignant')

In [ ]:
show_image_EI_MI(random9_ben[5:6], 'Benign')

In [ ]:
show_image_EI_MI(random9_ben[5:6], 'Benign')

In [ ]:
show_image_EI_MI(random9_ben[5:6], 'Benign')

In [ ]:
show_image_EI_MI(random9_mal[5:6], 'Malignant')

In [ ]:
show_image_EI_MI(random9_ben[5:6], 'Benign')

In [ ]:
show_image_EI_MI(random9_mal[5:6], 'Malignant')